Takes daily lethal heat hour count, and counts the number of lethal heat days per year.

This uses existing hours_over_lh files, which contain the daily number of hours per day over LH.
It fetches them from a remote bucket in batches.

Hour count files are output from cmip6_daily_lethal_heat_hours.ipynb

In [5]:
import xarray as xr
import matplotlib.pyplot as pyplot
import numpy as np
from os.path import join
import subprocess

In [36]:
# list of models to analyse and the name of the scenario
models = ['ACCESS-CM2','ACCESS-ESM1-5',
          'CNRM-CM6-1', 'CNRM-ESM2-1', 'CanESM5',
          'EC-Earth3-Veg-LR', 'FGOALS-g3', 'GFDL-CM4',
          'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR',
          'MIROC-ES2L', 'MIROC6', 'MPI-ESM1-2-HR',
          'MPI-ESM1-2-LR', 'MRI-ESM2-0']
scenario = 'ssp585'

# Directory names
dp_data = '/home/davidbyrne/disks/ssd2/lethal_heat/data/'
dp_out = join(dp_data,'ndays_pyear_by_year')
dp_tmp = join(dp_data, 'tmp/')
gs_holh = 'gs://fqqzlp/lethal_heat_vecellio/hours_over_lh_daily/'

In [ ]:
# Loop over models
for model in models:
    
    print(model, end='\r')

    # Template File names
    fp = join( dp_tmp, f'tolh_{model}_{scenario}_*.nc' )
    fp_out = join( dp_out, f'tolh_ndays_{model}_{scenario}.nc')
    
    # Download each model/scenario at a time
    fp_template = join(gs_holh, f'tolh_{model}_{scenario}_*.nc')
    subprocess.run(f'gsutil -m cp {fp_template} {dp_tmp}',
                   shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Open all files into one dataset
    ds = xr.open_mfdataset(fp, chunks='auto')['hours_over_lh']
    ds_out = xr.Dataset()
    ds_out['lon'] = ds['lon']
    ds_out['lat'] = ds['lat']
    
    # Set the number of hours to analyse
    hours = [3, 6, 9, 12, 15]
    for hh in hours:
        ds_out[f'ndays_over_{hh}hrs'] = (ds >= hh).resample(time='1Y').sum(dim='time')

    # Write to output file
    ds_out.to_netcdf(fp_out)
    
    # Remove temporary files
    subprocess.run(f'rm {dp_tmp}/*', shell=True)